In [1]:
import torch
import flwr as fl
import random
import numpy as np
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, Normalize, ToTensor
from client import get_client_generator, weighted_average_accuracy
from dataset import partition_dataset
from flwr.server.strategy import FedAvg
from flwr.simulation.backend.multiprocessing import MultiProcessingBackend
from flwr.simulation.backend.deterministic import SingleThreadedBackend
transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
train_dataset = MNIST("./mnist", train=True, download=True, transform=transform)
val_dataset = MNIST("./mnist", train=False, transform=transform)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

def seed_everything(seed):
    import random
    import numpy as np
    import torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def complete_run(seed=None, mode="ray"):
    if seed is not None:
        seed_everything(seed)

    num_clients = 6
    train_datasets = partition_dataset(train_dataset, 100)
    val_datasets = partition_dataset(val_dataset, 100)
    train_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) for dataset in train_datasets]
    val_dataloaders = [torch.utils.data.DataLoader(dataset, batch_size=16) for dataset in val_datasets]
    client_fn = get_client_generator(train_dataloaders, val_dataloaders)
    client_config = {
        "lr": 0.05,
        "epochs": 1,
    }
    strategy = FedAvg(
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        fraction_fit=0.5,
        fraction_evaluate=0.5,
        on_fit_config_fn=lambda _: client_config,
        on_evaluate_config_fn=lambda _: client_config,
        evaluate_metrics_aggregation_fn=weighted_average_accuracy,
    )
    if mode == "ray":
        backend = None
    elif mode == "multiprocessing":
        backend = MultiProcessingBackend()
    elif mode == "single_threaded":
        backend = SingleThreadedBackend()
    else:
        raise ValueError("Unknown mode")
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        seed_fn=seed_everything if seed is not None else None,
        seed=seed,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=5),
        strategy=strategy,
        backend=backend
    )

    return hist

/home/fredrik/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


./mnist/MNIST/raw/train-labels-idx1-ubyte
./mnist/MNIST/raw/t10k-labels-idx1-ubyte


In [21]:
import matplotlib.pyplot as plt

def plot_runs(runs):
    fig_loss, axs_loss = plt.subplots(1, len(runs), sharey=True)
    fig_acc, axs_acc = plt.subplots(1, len(runs), sharey=True)
    for i, run in enumerate(runs):
        axs_loss[i].plot([x[1] for x in run.losses_distributed])
        axs_acc[i].plot([x[1] for x in run.metrics_distributed["accuracy"]])
        if i>0:
            plt.setp(axs_loss[i].get_yticklabels(), visible=False)
            plt.setp(axs_acc[i].get_yticklabels(), visible=False)
    fig_loss.suptitle("Loss")
    fig_loss.set_size_inches(9,3)
    fig_loss.show()
    fig_acc.suptitle("Accuracy")
    fig_acc.set_size_inches(9,3)
    fig_acc.show()

In [3]:
# Failed determinism
seed_everything(0)
run1 = complete_run(None,"ray")
seed_everything(0)
run2 = complete_run(None,"ray")
seed_everything(0)
run3 = complete_run(None,"ray")
print(run1)
print(run2)
print(run3)
plot_runs([run1,run2,run3])

INFO flwr 2023-03-11 16:51:50,042 | app.py:199 | Starting Flower simulation, config: ServerConfig(num_rounds=5, round_timeout=None)
INFO flwr 2023-03-11 16:51:50,050 | server.py:100 | Initializing global parameters
INFO flwr 2023-03-11 16:51:50,051 | server.py:291 | Requesting initial parameters from one random client
INFO flwr 2023-03-11 16:51:50,059 | server.py:295 | Received initial parameters from one random client
INFO flwr 2023-03-11 16:51:50,060 | server.py:102 | Evaluating initial parameters
INFO flwr 2023-03-11 16:51:50,061 | server.py:115 | FL starting
DEBUG flwr 2023-03-11 16:51:50,062 | server.py:232 | fit_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-03-11 16:52:10,316 | server.py:246 | fit_round 1 received 2 results and 0 failures
WARNING flwr 2023-03-11 16:52:10,324 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-03-11 16:52:10,325 | server.py:179 | evaluate_round 1: strategy sampled 2 clients (out of 5)
DEBUG flwr 2023-03-11 16:

History (loss, distributed):
	round 1: 0.23897419881820678
	round 2: 0.14989428592845797
	round 3: 0.1124828438712284
	round 4: 0.09364412524877117
	round 5: 0.08764808896835893
History (metrics, distributed):
{'accuracy': [(1, 0.93275), (2, 0.95475), (3, 0.966), (4, 0.97025), (5, 0.97075)]}
History (loss, distributed):
	round 1: 0.2513150454834104
	round 2: 0.14717471213638783
	round 3: 0.10913577037304639
	round 4: 0.09507927848678083
	round 5: 0.08463426199601963
History (metrics, distributed):
{'accuracy': [(1, 0.9345), (2, 0.9545), (3, 0.96925), (4, 0.969), (5, 0.97325)]}
3min 34s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
# Successful determinism
run1 = complete_run(0,"ray")
run2 = complete_run(0,"ray")
run3 = complete_run(0,"ray")
print(run1)
print(run2)
print(run3)
plot_runs([run1,run2,run3])